In [ ]:
%pip install ipywidgets --upgrade
%pip install jupyter --upgrade

In [ ]:
%pip install transformers torch

In [1]:
from konlpy.tag import Okt
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

c:\ProgramData\anaconda3\envs\fastapi_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# from konlpy.tag import Okt
# from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# 형태소 분석기 로드
okt = Okt()

# 학습된 모델 로드
model_name = "../model/my_trained_model/"  # 학습된 모델이 저장된 디렉토리 경로
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)

# 텍스트 전처리 함수
def preprocess_text(text):
    return text.replace('며', '. ').replace('고', '. ').replace('이라', '. ')

# 문장 유사도 평가 함수 (TF-IDF 기반)
def evaluate_context_similarity(prev_sentence, current_sentence):
    vectorizer = TfidfVectorizer().fit_transform([prev_sentence, current_sentence])
    vectors = vectorizer.toarray()
    similarity = cosine_similarity([vectors[0]], [vectors[1]])
    return similarity[0][0]

# 중복 문장 필터링 (유사도를 기반으로 중복 문장 제거)
def remove_duplicate_sentences(summary):
    sentences = summary.split('. ')
    unique_sentences = []
    
    for i in range(len(sentences)):
        if i == 0:  # 첫 문장은 무조건 추가
            unique_sentences.append(sentences[i])
        else:
            prev_sentence = sentences[i - 1]
            current_sentence = sentences[i]
            similarity = evaluate_context_similarity(prev_sentence, current_sentence)
            if similarity < 0.8:  # 유사도가 0.8 미만일 경우만 추가
                unique_sentences.append(current_sentence)

    return '. '.join(unique_sentences)

# 종결어미 목록
final_endings = ['다', '요', '라', '네', '죠', '자']

# 문장 완성 체크 함수
def ensure_sentence_completion(sentence):
    if not any(sentence.strip().endswith(ending) for ending in final_endings):
        sentence += '다.'  # 종결어미가 없으면 강제로 완성
    return sentence

# 불완전한 문장 필터링 및 문장 완성 처리
def process_summary(summary):
    sentences = summary.split('. ')
    complete_sentences = []
    
    for i, sentence in enumerate(sentences):
        sentence = sentence.strip()
        if sentence:
            # 중복된 문장은 자동으로 제거되므로, 문장 완성도를 체크하여 보완
            sentence = ensure_sentence_completion(sentence)
            complete_sentences.append(sentence)

    return '. '.join(complete_sentences)

# 요약 생성 함수
def generate_summary(text, model, tokenizer):
    inputs = tokenizer([text], max_length=512, truncation=True, return_tensors="pt")
    summary_ids = model.generate(
        inputs['input_ids'], 
        max_length=300,  # 요약의 최대 길이를 증가
        min_length=80,   # 최소 길이도 증가
        length_penalty=1.2,  # 길이 패널티 조정
        num_beams=6,  # 빔 서치 개수 유지
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 요약 적용 함수 (전체 워크플로우)
def summarize_article(article_text):
    # 텍스트 전처리
    preprocessed_text = preprocess_text(article_text)
    
    # 요약 생성
    summary = generate_summary(preprocessed_text, model, tokenizer)
    
    # 요약 후처리 (중복 제거 및 문장 완성)
    final_summary = process_summary(summary)
    
    return final_summary

# 예시 기사
input_text = """
[서울경제]

인공지능(AI) 솔루션 기업 BHSN이 기업의 계약관리 혁신을 위해 PwC컨설팅과 전략적 파트너십을 체결했다고 21일 밝혔다.

양사는 이번 파트너십을 통해 BHSN의 비즈니스·리걸 특화 AI 기술과 계약관리솔루션 '앨리비' 고도화에 협업한다. 기업에 제공할 수 있는 서비스의 범위를 확대하고 기업이 안전하게 비즈니스를 영위할 수 있도록 돕는 전문 서비스를 제공해 나갈 예정이다.

앨리비는 BHSN이 자체 기술로 개발한 올인원 AI 비즈니스 솔루션이다. 계약의 작성, 검토, 조율 및 합의, 승인, 서명, 보관 및 AI 검색까지 기업 내 모든 부서에서 필요로 하는 통합 계약 프로세스를 제공하고 있다.

임정근 BHSN 대표는 “비즈니스 의사결정에 반드시 필요한 계약은 제대로 관리되지 않을 경우 기업에 큰 리스크를 끼칠 수 있는 아주 중요한 사안”이라며 “적극적 양사 협업을 통해 국내 기업들이 더욱 안전하게 계약을 관리하고 비즈니스를 영위할 수 있도록 힘을 아끼지 않을 것”이라고 말했다.

차경민 PwC컨설팅 파트너는 “BHSN은 비즈니스와 리걸에 특화된 AI 생성 및 검색 기술을 보유하고, 기술을 기반으로 자체 서비스형소프트웨어(SaaS) 솔루션까지 개발해 제공하는 준비된 기업”이라며 “양사 파트너십을 통해 PwC컨설팅의 전문 컨설팅과 함께 솔루션을 통해 프로세스를 혁신할 수 있는 구체적 서비스까지 제공할 예정”이라고 전했다.

김성태 기자(kim@sedaily.com)
"""

# 기사 요약 실행
summarized_text = summarize_article(input_text)
print(summarized_text)


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


인공지능(AI) 솔루션 기업 BHSN이 PwC컨설팅과 전략적 파트너십을 체결했다고 21일 밝혔다. BHSN은 이번 파트너십을 통해 비즈니스·리걸 특화 AI 기술과 계약관리솔루션 '앨리비' 도화에 협업한다. BHSN은 자체 기술로 개발한 올인원 AI 비즈니스 솔루션인앨리비를 자체 기술로 개발했다. 임 BHSN 대표는 “비즈니스 의사결정에 반드시 필요한 계약은 제대로 관리되지 않을 경우 기업에 큰 리스크를 끼칠 수 있는 아주 중요한 사안”이라며 “양사 협업을 통해 국내 기업들이 더욱 안전하게 계약을 관리하고 비즈니스를 영위할 수 있도록 힘을 아끼지 않을 것”이라고 말했다


- 위 코드 실행시간 17:12:55 ~ 17:14:35 1분 40초 소요

- ' BHSN은 자체 기술로 개발한 올인원 AI 비즈니스 솔루션인앨리비를 자체 기술로 개발했다.' 부분의 중복된 내용 , 
- ''앨리비' 도화' -> ''앨리비' 고도화' 로의 오타 및 부정확한 단어 수정,
- '계약관리 혁신에 대한 내용 강조의 부재 ('서비스 범위 확대', '안전한 비즈니스 지원' 등)

#### 위의 3가지를 보완하면 더 좋은 퀄리티의 요약문을 뽑아낼 수 있을 것임 !

##  ~2024.10.21

In [3]:
import requests

In [9]:
# 요약된 데이터를 
url = 'http://127.0.0.1:8000/api/summarization/upload_summary'
# 요약된 텍스트 정의
summary_data = {
    'text' : summarized_text
}

# POST 요청으로 엔드포인트에 데이터 전송
response = requests.post(url, json=summary_data)

# 응답 출력
if response.status_code == 200:
    print("요약 텍스트가 성공적으로 업로드되었습니다:", response.json())
else:
    print("요약 텍스트 업로드에 실패했습니다:", response.status_code, response.text)

요약 텍스트가 성공적으로 업로드되었습니다: {'summaryText': "인공지능(AI) 솔루션 기업 BHSN이 PwC컨설팅과 전략적 파트너십을 체결했다고 21일 밝혔다. BHSN은 이번 파트너십을 통해 비즈니스·리걸 특화 AI 기술과 계약관리솔루션 '앨리비' 도화에 협업한다. BHSN은 자체 기술로 개발한 올인원 AI 비즈니스 솔루션인앨리비를 자체 기술로 개발했다. 임 BHSN 대표는 “비즈니스 의사결정에 반드시 필요한 계약은 제대로 관리되지 않을 경우 기업에 큰 리스크를 끼칠 수 있는 아주 중요한 사안”이라며 “양사 협업을 통해 국내 기업들이 더욱 안전하게 계약을 관리하고 비즈니스를 영위할 수 있도록 힘을 아끼지 않을 것”이라고 말했다"}
